# Text Generation

In [1]:
# importing packages & libraries
import tensorflow as tf
import numpy as np

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

## Get reference corpus

In [2]:
# reading and processing data
data = open('./data/irish-lyrics.txt').read()
corpus = data.lower().split("\n")

In [3]:
# defining tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# print(tokenizer.word_index)
print(total_words)

2690


## Preprocess

In [4]:
# preprocessing data
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# padding sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# creating predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

## Create LSTM Model

In [5]:
# creating model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=max_sequence_len-1),
    tf.keras.layers.Bidirectional(LSTM(150)),
    tf.keras.layers.Dense(total_words, activation = 'softmax'),
])

model.compile(loss='categorical_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])

In [6]:
# training model
history = model.fit(xs, ys, epochs=10, verbose=1)

Train on 12038 samples
Epoch 1/10
12038/12038 [==============================] - 10s 855us/sample - loss: 6.7406 - accuracy: 0.0654
Epoch 2/10
12038/12038 [==============================] - 8s 705us/sample - loss: 6.2529 - accuracy: 0.0737
Epoch 3/10
12038/12038 [==============================] - 9s 714us/sample - loss: 5.9852 - accuracy: 0.0835
Epoch 4/10
12038/12038 [==============================] - 9s 711us/sample - loss: 5.6800 - accuracy: 0.0998
Epoch 5/10
12038/12038 [==============================] - 9s 708us/sample - loss: 5.3554 - accuracy: 0.1146
Epoch 6/10
12038/12038 [==============================] - 9s 714us/sample - loss: 5.0215 - accuracy: 0.1334
Epoch 7/10
12038/12038 [==============================] - 9s 708us/sample - loss: 4.6793 - accuracy: 0.1502
Epoch 8/10
12038/12038 [==============================] - 8s 700us/sample - loss: 4.3419 - accuracy: 0.1710
Epoch 9/10
12038/12038 [==============================] - 9s 714us/sample - loss: 4.0007 - accuracy: 0.2043
Epoc

## Generate text sequences

In [7]:
# generating text
seed_text = "I too am a human being and I"
# seed_text = "I've got a bad feeling about this"
next_words = 100
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

I too am a human being and I was seen in the stony sea of the upwards wind of doneen sends by the sea they treads the upwards church dwell near fulfill you more young young young young more young more young more grey our hand and our hat and they did stand and shrill they did stand in side and tory ghosts and stainless they did say me now dhu easy and the wind of spancil hill and tory ghosts and goblins lingers now now writin to go street i sat side by corporal casey by spancil hill today today by the clay clay turning treads the
